Related page: https://model.earth/data-commons/docs/data

TO DO: Reduce objust to just state number 01 instead of geoId/01

In [ ]:
pip install datacommons_pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 952.4 kB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import datacommons_pandas as dc
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests

In [ ]:
stateDict = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California", "CO": "Colorado",
    "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana",
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota",
    "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming",
    "DC": "District of Columbia",
    # US Territories
    "AS": "American Samoa", "GU": "Guam", "MP": "Northern Mariana Islands", "PR": "Puerto Rico", "VI": "Virgin Islands of the U.S."
}

stateData = pd.DataFrame(list(stateDict.items()),columns = ['State','StateName'])
stateData.head()

,State,StateName
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [ ]:
def get_state_ids():
  # Getting the states  dcids
  state_dcids = dc.get_places_in(['country/USA'], 'State')
  # Getting the names of the states
  state_names = [dc.get_property_values([name], 'name') for name in state_dcids['country/USA']]

  data = {}
  for entry in state_names:
      for k,v in entry.items():
          data[k] = v[0]

  # Adding Rest 4 US Territories
  for k,v in dc.get_property_values(['geoId/60','geoId/66','geoId/69','geoId/78'], 'name').items():
      data[k] = v[0]
  return data

In [ ]:
state_ids_data = get_state_ids()
state_ids_data

{'geoId/01': 'Alabama',
 'geoId/02': 'Alaska',
 'geoId/04': 'Arizona',
 'geoId/05': 'Arkansas',
 'geoId/06': 'California',
 'geoId/08': 'Colorado',
 'geoId/09': 'Connecticut',
 'geoId/10': 'Delaware',
 'geoId/11': 'District of Columbia',
 'geoId/12': 'Florida',
 'geoId/13': 'Georgia',
 'geoId/15': 'Hawaii',
 'geoId/16': 'Idaho',
 'geoId/17': 'Illinois',
 'geoId/18': 'Indiana',
 'geoId/19': 'Iowa',
 'geoId/20': 'Kansas',
 'geoId/21': 'Kentucky',
 'geoId/22': 'Louisiana',
 'geoId/23': 'Maine',
 'geoId/24': 'Maryland',
 'geoId/25': 'Massachusetts',
 'geoId/26': 'Michigan',
 'geoId/27': 'Minnesota',
 'geoId/28': 'Mississippi',
 'geoId/29': 'Missouri',
 'geoId/30': 'Montana',
 'geoId/31': 'Nebraska',
 'geoId/32': 'Nevada',
 'geoId/33': 'New Hampshire',
 'geoId/34': 'New Jersey',
 'geoId/35': 'New Mexico',
 'geoId/36': 'New York',
 'geoId/37': 'North Carolina',
 'geoId/38': 'North Dakota',
 'geoId/39': 'Ohio',
 'geoId/40': 'Oklahoma',
 'geoId/41': 'Oregon',
 'geoId/42': 'Pennsylvania',
 'geo

In [ ]:
# CO2 = Annual_Emissions_CarbonDioxide_NonBiogenic -- State
# Methane = Annual_Emissions_Methane_NonBiogenic -- State
# Population = Count_Person -- State

def fetch_timelines_data():
  # Getting DCIDs for all the states
  state_ids_data = get_state_ids()
  # Getting the timelines data
  timelines_data = {}
  for key,value in state_ids_data.items():
    try:
      timelines_data[value] = {'Population':round(dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']['2022'],2)}
    except:
      timelines_data[value] = {'Population':0.00}
    try:
      timelines_data[value].update({'CO2':round(dc.get_stat_all([key], ['Annual_Emissions_CarbonDioxide_NonBiogenic'])[key]['Annual_Emissions_CarbonDioxide_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'CO2':0.00})
    try:
      timelines_data[value].update({'Methane':round(dc.get_stat_all([key], ['Annual_Emissions_Methane_NonBiogenic'])[key]['Annual_Emissions_Methane_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'Methane':0.00})

  # Creating the dataframe
  normalized_data = []
  for state, values in timelines_data.items():
      entry = {'StateName': state}
      entry.update(values)
      normalized_data.append(entry)
  dataframe = pd.DataFrame(normalized_data)

  # Adding state abbreviations to the dataframe
  dataframe = stateData.merge(dataframe,how = 'inner',on='StateName')

  # Dividing the data by 1000 and rounding to 2 decimals
  # for col in dataframe.iloc[:,2:]:
  #     dataframe[col] = round(dataframe[col]/1000,2)

  return dataframe

In [ ]:
timelines_df = fetch_timelines_data()
timelines_df.head()

,State,StateName,Population,CO2,Methane
0,AL,Alabama,5074296.00,68215710.10,7657440.42
1,AK,Alaska,733583.00,16939106.60,871681.75
2,AZ,Arizona,7359197.00,38392400.30,1361141.25
3,AR,Arkansas,3045637.00,36628789.10,2123889.00
4,CA,California,39029342.00,91754567.10,8737629.75


In [ ]:
# Fetching the data for land area of the states
area_data = gpd.read_file(f'https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip')
area_data["SqMiles"] = round(area_data['ALAND']/ 2589988.110336,2)
area_data.head()

,STATEFP,STATENS,GEOIDFQ,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,SqMiles
0,35,00897535,0400000US35,35,NM,New Mexico,00,314198587197,726463919,"POLYGON ((-109.05017 31.48000, -109.04984 31.4...",121312.75
1,46,01785534,0400000US46,46,SD,South Dakota,00,196341525171,3387709166,"POLYGON ((-104.05788 44.99761, -104.05078 44.9...",75807.89
2,06,01779778,0400000US06,06,CA,California,00,403673296401,20291770234,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",155859.13
3,21,01779786,0400000US21,21,KY,Kentucky,00,102266598312,2384223544,"MULTIPOLYGON (((-89.40565 36.52817, -89.39869 ...",39485.35
4,01,01779775,0400000US01,01,AL,Alabama,00,131185049346,4582326383,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",50650.83


In [ ]:
timelines_df = timelines_df.merge(area_data[['STUSPS','SqMiles']], how='inner', left_on='State', right_on='STUSPS')
timelines_df.drop(columns = ['STUSPS'],inplace=True)
timelines_df

,State,StateName,Population,CO2,Methane,SqMiles
0,AL,Alabama,5074296.00,68215710.10,7657440.42,50650.83
1,AK,Alaska,733583.00,16939106.60,871681.75,571051.62
2,AZ,Arizona,7359197.00,38392400.30,1361141.25,113655.39
3,AR,Arkansas,3045637.00,36628789.10,2123889.00,51992.70
4,CA,California,39029342.00,91754567.10,8737629.75,155859.13
5,CO,Colorado,5839926.00,50334501.47,8793630.94,103637.06
6,CT,Connecticut,3626205.00,10812606.90,309106.88,4842.40
7,DE,Delaware,1018396.00,5142434.00,401666.25,1948.54
8,FL,Florida,22244823.00,102634265.80,6603958.08,53654.21
9,GA,Georgia,10912876.00,48611013.40,5377826.75,57716.60


In [ ]:
# Converting to a CSV file
timelines_df.to_csv('UN_Timelines_Data.csv', index=False)